# Project 04: Spatial Data Analysis and Geoprocessing Pipeline

This project demonstrates advanced spatial data analysis using geospatial Python libraries. It covers:
- Loading and processing GeoJSON spatial datasets
- Buffer analysis and spatial joins
- Coordinate reference system (CRS) transformations
- Road network extraction from OpenStreetMap
- Geospatial aggregation and visualization
- Automated export of geoprocessed results

## Dataset
Multiple GeoJSON files representing points, roads, and country boundaries, plus OpenStreetMap data for road network analysis.

## 1. Import Libraries and Setup

In [1]:
import geopandas as gpd
import pyrosm
import json
import matplotlib.pyplot as plt
from shapely.ops import linemerge
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = Path('data')
OUTPUT_PATH = Path('output')
OUTPUT_PATH.mkdir(exist_ok=True)

print("Libraries imported successfully")

Libraries imported successfully


## 2. Load Parameters and GeoJSON Data

In [2]:
with open(DATA_PATH / 'proj4_params.json', 'r') as params_file:
    params = json.load(params_file)
print(params)

points = gpd.read_file(DATA_PATH / 'proj4_points.geojson')
print(points.head())

{'city': 'Cracow', 'city_pyrosm': 'Cracow', 'city_osmnx': 'KrakĂłw, Poland', 'id_column': 'lamp_id'}
   lamp_id circuit       label                         geometry
0     5907    4260  4260 III/9  POINT (2215161.628 6459108.749)
1     5908    4104    4104 I/7  POINT (2214299.747 6459283.830)
2     5909    4070    4070 I/8  POINT (2215058.617 6458937.362)
3     5910    4148   4148 I/14  POINT (2214001.935 6459017.980)
4     5911    4148    4148 I/4  POINT (2214331.397 6458976.290)


## 3. Buffer Analysis and Spatial Join

In [3]:
lamp_id = points['lamp_id']
b_points = points.copy()
b_points['geometry'] = b_points['geometry'].buffer(155.8)
joined = gpd.sjoin(points, b_points, predicate='intersects')
counts = joined.groupby(lamp_id).size().reset_index(name='count')
counts.to_csv(OUTPUT_PATH / 'proj4_ex01_counts.csv', index=False)

points2 = points.copy().to_crs(epsg=4326)
points2['lat'] = points2.geometry.y.round(7)
points2['lon'] = points2.geometry.x.round(7)
points2 = points2[['lamp_id', 'circuit', 'label', 'lat', 'lon']]
coords_df = points2[['lamp_id', 'lat', 'lon']]
coords_df.to_csv(OUTPUT_PATH / 'proj4_ex01_coords.csv', index=False)
print(counts.head())

   lamp_id  count
0     5907     16
1     5908     16
2     5909     17
3     5910     20
4     5911      9


## 4. Road Network Extraction from OpenStreetMap

In [4]:
fp = pyrosm.get_data('Cracow')
osm = pyrosm.OSM(fp)
gdf_driving = osm.get_network(network_type='driving')
gdf_filtered = gdf_driving[gdf_driving['highway'] == 'tertiary']
gdf_filtered['geometry'] = gdf_filtered.geometry.apply(lambda x: linemerge(x) if x.geom_type == 'MultiLineString' else x)
gdf_filtered.rename(columns={'id': 'osm_id'}, inplace=True)
gdf_filtered[['osm_id', 'name', 'geometry']].to_file(OUTPUT_PATH / 'proj4_ex02_roads.geojson', driver='GeoJSON')
gdf_filtered.to_csv(OUTPUT_PATH / 'proj4_ex02_gdf_filtered.csv', index=False)
print(gdf_filtered.head())

Downloaded Protobuf data 'Cracow.osm.pbf' (29.47 MB) to:
'C:\Users\kniko\AppData\Local\Temp\pyrosm\Cracow.osm.pbf'
         access  area bicycle bridge cycleway est_width  foot footway  \
1          None  None    None   None     None      None  None    None   
9          None  None    None   None     None      None  None    None   
10         None  None    None   None     None      None  None    None   
11         None  None    None   None     None      None  None    None   
17  destination  None    None   None     None      None  None    None   

     highway int_ref  ... tunnel  turn width   osm_id timestamp version  \
1   tertiary    None  ...   None  None  None  2954556         0       0   
9   tertiary    None  ...   None  None  None  4757981         0       0   
10  tertiary    None  ...   None  None  None  4758409         0       0   
11  tertiary    None  ...   None  None  None  4758410         0       0   
17  tertiary    None  ...   None  None  None  5095912         0       0

## 5. CRS Transformation and Buffer Join

In [5]:
roads = gpd.read_file(OUTPUT_PATH / 'proj4_ex02_roads.geojson').to_crs('epsg:2178')
points = gpd.read_file(DATA_PATH / 'proj4_points.geojson').to_crs('epsg:2178')
roads_buf = roads.copy()
roads_buf['geometry'] = roads.geometry.buffer(50, cap_style=2)
joined333 = gpd.sjoin(points, roads_buf, predicate='intersects')
street_point_counts = joined333.groupby('name').size().reset_index(name='point_count')
street_point_counts = street_point_counts[street_point_counts['point_count'] > 0]
street_point_counts.to_csv(OUTPUT_PATH / 'proj4_ex03_streets_points.csv', index=False)
print(street_point_counts.head())

                               name  point_count
0                      Aleja 3 Maja          131
1                    Aleja Kijowska          128
2  Aleja Marszałka Ferdinanda Focha           32
3                           Balicka           80
4              Bartosza Głowackiego           22


## 6. Country Boundaries Visualization

In [6]:
countries = gpd.read_file(DATA_PATH / 'proj4_countries.geojson')
countries['geometry'] = countries['geometry'].boundary
countries.to_pickle(OUTPUT_PATH / 'proj4_ex04_gdf.pkl')
for idx, country_name in enumerate(countries['name']):
    img_filename = OUTPUT_PATH / f'proj4_ex04_{country_name.lower().replace(" ", "_")}.png'
    fig, ax = plt.subplots(figsize=(10, 10))
    countries.iloc[[idx]].plot(ax=ax, edgecolor='blue', linewidth=1)
    plt.savefig(img_filename)
    plt.close(fig)
print('Country boundary images saved.')

Country boundary images saved.
